In [2]:
import luxtelligence_lnoi400_forge as lxt
import numpy as np
import photonforge as pf
import tidy3d as td
from photonforge import live_viewer

viewer = live_viewer.LiveViewer(port=5001)

Starting live viewer at http://localhost:5001


In [3]:
tech = lxt.lnoi400(include_substrate=False, include_top_opening=False)
pf.config.default_technology = tech

port_spec = tech.ports["RWG1000"]

pf.config.default_kwargs = {
    "port_spec": port_spec,
    "radius": 75,
    "euler_fraction": 0.5,
}

In [4]:
@pf.parametric_component
def termination(*, length, offset_factor=0.3, sign=1):
    c = pf.Component()

    for layer, path in port_spec.get_paths((0, 0)):
        w = None
        l = length

        if layer == tech.layers["LN_RIDGE"].layer:
            w = pf.config.tolerance
            l -= 0.5 * port_spec.width

        c.add(
            layer, path.bezier([(0.5 * l, 0), (l, sign * offset_factor * l)], width=w)
        )

    c.add_port(pf.Port((0, 0), 0, port_spec))

    # We use an analitical termination model to remove all reflections, but optimization
    # of the design could be done with a Tidy3D model.
    c.add_model(pf.TerminationModel())
    return c


terminations = [
    termination(length=2 * pf.config.default_kwargs["radius"], sign=-1),
    termination(length=2 * pf.config.default_kwargs["radius"], sign=1),
]

viewer(terminations[0])

In [ ]:
@pf.parametric_component
def mzi(
    *,
    added_length=0,
    stages=1,
    upper_arm="wg",
    lower_arm="ps",
    output_mmi=True,
):

    c = pf.Component()

    # Phase-shifter arm
    ps = phase_shifter()
    ps_len = ps.parametric_kwargs["length"]

    # Bare waveguide arm
    wg = pf.parametric.straight(length=ps_len)

    # Minimal arm offsets and distance w.r.t. the MMIs
    offset = max(2 * ps.parametric_kwargs["heater_pad_size"], port_spec.width)

    if added_length == 0:
        # Balanced case: use S bends for routing
        route = pf.parametric.route_s_bend
        distance = 6 * offset
        # Offsets for lower and upper arms
        offsets = [-offset, offset]
    else:
        # Increase offset to guarantee that no S bends are used in the route,
        # which, in turn, guarantees that the added length is exact.
        distance = 2 * pf.config.default_kwargs["radius"]
        offset = max(offset, distance + pf.config.tolerance)
        route = pf.parametric.route
        # Offsets for lower and upper arms
        offsets = [min(0, added_length / 2) - offset, max(0, added_length / 2) + offset]

    # Calculate position to align MMIs at the x axis
    x0 = mmi.ports["P0"].center[0]
    y0 = 0.5 * (mmi.ports["P1"].center[1] + mmi.ports["P0"].center[1])

    # For multi-stage, calculate spacing
    mmi_len = mmi.ports["P2"].center[0] - mmi.ports["P0"].center[0]
    spacing = (mmi_len + ps_len + 2 * distance, 0)
    mmi_cols = (stages + 1) if output_mmi else stages

    # Add all MMIs
    mmi_ref = pf.Reference(mmi, origin=(-x0, -y0), columns=mmi_cols, spacing=spacing)
    c.add(mmi_ref)

    # Add ports. If no output MMI, add output ports from arms later.
    c.add_port([mmi_ref.get_ports("P0")[0], mmi_ref.get_ports("P1")[0]])
    if output_mmi:
        c.add_port([mmi_ref.get_ports("P2")[-1], mmi_ref.get_ports("P3")[-1]])

    # Arm start position
    x_ps = mmi_ref.get_ports("P2")[0].center[0] + distance

    # Add each arm to all stages
    for i, arm in enumerate([lower_arm, upper_arm]):
        arm = {"wg": wg, "ps": ps}.get(arm)
        if arm is None:
            # Use terminations on both MMIs
            mmi_port = mmi_ref.get_ports(f"P{2 + i}")[0]
            term_ref = pf.Reference(terminations[i], columns=stages, spacing=spacing)
            term_ref.connect("P0", mmi_port)
            c.add(term_ref)

            # There's only 1 MMI for a single stage without output MMI
            if mmi_cols > 1:
                mmi_port = mmi_ref.get_ports(f"P{i}")[1]
                term_ref = pf.Reference(
                    terminations[1 - i], columns=mmi_cols - 1, spacing=spacing
                )
                term_ref.connect("P0", mmi_port)
                c.add(term_ref)
        else:
            arm = pf.Reference(arm, (x_ps, offsets[i]), columns=stages, spacing=spacing)
            rt = pf.Reference(
                route(port1=(mmi_ref, f"P{2 + i}", 0), port2=(arm, "P0", 0)),
                columns=stages,
                spacing=spacing,
            )
            c.add(arm, rt)

            # Route to next stage only when there is a second MMI
            if mmi_cols > 1:
                rt = pf.Reference(
                    route(port1=(mmi_ref, f"P{i}", 1), port2=(arm, "P1", 0)),
                    columns=mmi_cols - 1,
                    spacing=spacing,
                )
                c.add(rt)

            # Add output port if no output MMI is used
            if not output_mmi:
                c.add_port(arm.get_ports("P1")[-1])

            # Add terminals, if using a phase-shifter
            if len(arm.component.terminals) > 0:
                arm_terminals = arm.get_terminals()
                c.add_terminal(
                    [
                        arm_terminals[name][index]
                        for index in range(stages)
                        for name in ("T0", "T1")
                    ]
                )

    c.add_model(pf.CircuitModel())

    return c


viewer(mzi())